# The Problem



## Initial Energy Terms

Here we initialize the energy terms, defined by the total energy equation:

![alt text](https://image.slidesharecdn.com/molecularmodellingforinsilicodrugdiscovery-141212161303-conversion-gate01/95/molecular-modelling-for-in-silico-drug-discovery-17-638.jpg?cb=1454359395)

In our model, we introduce some other energy terms to this energy equation, as shown in the next code block:

In [77]:
import numpy as np
import random as random

# returns a vector that has all the energy terms 
def energy_terms():
    # initial values of the energy terms
    bond = 49.7829
    angle = 138.4463
    dihed = 217.8041
    nb = 39.1752
    eel = 695.8385
    vdw = -83.5197
    el = -1162.1836
    egb = -162.3717
    
    return np.array([bond, angle, dihed, nb, eel, vdw, el, egb])

# https://docs.scipy.org/doc/numpy/reference/generated/numpy.array.html
    
print(sum(energy_terms()))

-267.028


## Consider the weights in the equation

In [10]:
# initialize all the weights to 1
wb = 1
wa = 1
wd = 1
wn = 1
wee = 1
wv = 1
wel = 1
weg = 1

weights = [wb,wa,wd,wn,wee,wv,wel,weg]

# returns a vector describing all of the weights
def weights():
    return [wb,wa,wd,wn,wee,wv,wel,weg]

print(sum(weights()))

8


The sum of these weights is equal to 8. We want to keep this sum total weight equal to 8 as we perform our sensitivity analysis, while redistributing the individual weights.

In [17]:
def total_energy():    
    return wb*bond + wa*angle + wd*dihed + wn*nb + wee*eel + wv*vdw + wel*el + weg*egb

print(total_energy())

-267.02800000000013


# Energy Minimization via Sensitivity Analysis

The objective here is to minimize the total energy via the process of sensitivity analysis. In other words, we will vary the weights of each of the energy terms, while keeping the sum total weights constant, so that we return the lowest value of total energy.

Here we use the __one-at-a-time__ technique known in the field of sensitivty analysis, to vary the weights individually, one at a time.

## Genetic Algorithm

Instead of using the one-at-a-time method, I have chosen to try a different method. 

Here we apply the metaheuristic, a genetic algorithm, to solve the energy minimization problem by redistributing the weights such that the total energy equation is minimized.

In [203]:
# random initialization of an attempt at the solution, an "individual"
individual = []

# total amount of weight left to be redistributed 
total_weight_left = float(len(weights()))
print(total_weight_left)

# distribute random weights to an individual, while keeping total weight constant
for weight in (range(len(weights())-1)):    
    distribute = random.uniform(0,1) * total_weight_left
    total_weight_left -= distribute
    new_weight = distribute
    individual.append(new_weight)
    
# add what's left to be distributed, as the final weight 
individual.append(total_weight_left)

# total weight is unchanged 
# weights have been randomly distributed
print(individual)
print(sum(individual))

"""initialize energy terms"""

bond = 49.7829
angle = 138.4463
dihed = 217.8041
nb = 39.1752
eel = 695.8385
vdw = -83.5197
el = -1162.1836
egb = -162.3717

energy_terms = np.array([[bond], [angle], [dihed], [nb], [eel], [vdw], [el], [egb]])

"""matrix multiplication to computer total_energy"""

np.array(individual)

# calculate dot product of the arrays
total_energy = np.dot(individual, energy_terms)

print(total_energy)

# repeat this, but with weights all equal to 1 just to verfiy that it's working
print(np.dot(np.ones(len(energy_terms)), energy_terms))

8.0
[6.131103168674101, 1.0396547403942122, 0.7556627445569424, 0.06360118699589173, 0.0011069394528320745, 0.0011527028117522583, 0.0006611989796957597, 0.007057318134572676]
8.0
[ 614.99811557]
[-267.028]


##### Potential Problem

It is worth noting that this method of distributing weights will favour the generated weights at the start of the for loop. The weights towards the end of the list will be closer to zero, and the weights at the start of the individual's list of weights will be closer to 8, as it has a greater chance at getting the total weight of 8, as there is more total weight left to choose from.

To address this problem, I shuffle the weights in the next section, but perhaps more can be done to improve this.

In [209]:
population = []

# random initialization of an attempt at the solution, an "individual"
individual = []

# total amount of weight left to be redistributed 
total_weight_left = float(len(weights()))
# print(total_weight_left)

# distribute random weights to an individual, while keeping total weight constant
for weight in (range(len(weights())-1)):    
    distribute = random.uniform(0,1) * total_weight_left
    total_weight_left -= distribute
    new_weight = distribute
    individual.append(new_weight)
    
# add what's left to be distributed, as the final weight 
individual.append(total_weight_left)

# total weight is unchanged 
# weights have been randomly distributed

# randomly shuffle the weights in the list
random.shuffle(individual)
print_weights_of(individual)

# print(individual)
# print(sum(individual))

"""initialize energy terms"""

bond = 49.7829
angle = 138.4463
dihed = 217.8041
nb = 39.1752
eel = 695.8385
vdw = -83.5197
el = -1162.1836
egb = -162.3717

energy_terms = np.array([[bond], [angle], [dihed], [nb], [eel], [vdw], [el], [egb]])

"""matrix multiplication to computer total_energy"""

np.array(individual)

# calculate dot product of the arrays
total_energy = np.dot(individual, energy_terms)

# original weights of 1
original_energy = np.dot(np.ones(len(energy_terms)), energy_terms)

# print sum of all the weights in this attempt at the solution, notice it is constant
print(sum(individual))
print(individual)
print(original_energy)
print(total_energy)
# the objective is to return the most negative total_energy


Here are the weights for the following energy terms...

 bond = 0.019092348856810923
 angle = 0.04041668748175255
 dihed = 0.022182191513807733
 nb = 0.00523848843796337
 eel = 0.10572708745745141
 vdw = 0.013678634950055493
 el = 7.738814013447898
 egb = 0.05485054785426009

8.0
[0.019092348856810923, 0.04041668748175255, 0.022182191513807733, 0.00523848843796337, 0.10572708745745141, 0.013678634950055493, 7.738814013447898, 0.05485054785426009]
[-267.028]
[-8918.8197597]


## Incomplete, but working Genetic Algorithm

This code can be made much more efficient by using the subroutines involved in a genetic algorithm:
* mutation
* crossover
* tournament selection

However, this piece of code __does work__ as a kind of heuristic to this problem. If you change the value of end to higher value, you will eventually reach the solution with enough accuracy.

In [221]:
# original weights of 1
original_energy = np.dot(np.ones(len(energy_terms)), energy_terms)
last_energy = original_energy
end = 100

for a in range(end):

    population = []

    # random initialization of an attempt at the solution, an "individual"
    individual = []

    # total amount of weight left to be redistributed 
    total_weight_left = float(len(weights()))
    # print(total_weight_left)

    # distribute random weights to an individual, while keeping total weight constant
    for weight in (range(len(weights())-1)):    
        distribute = random.uniform(0,1) * total_weight_left
        total_weight_left -= distribute
        new_weight = distribute
        individual.append(new_weight)

    # add what's left to be distributed, as the final weight 
    individual.append(total_weight_left)

    # total weight is unchanged 
    # weights have been randomly distributed

    # randomly shuffle the weights in the list
    random.shuffle(individual)

    """initialize energy terms"""

    bond = 49.7829
    angle = 138.4463
    dihed = 217.8041
    nb = 39.1752
    eel = 695.8385
    vdw = -83.5197
    el = -1162.1836
    egb = -162.3717

    energy_terms = np.array([[bond], [angle], [dihed], [nb], [eel], [vdw], [el], [egb]])

    """matrix multiplication to computer total_energy"""

    np.array(individual)

    # calculate dot product of the arrays
    total_energy = np.dot(individual, energy_terms)

    if total_energy < last_energy:
        last_energy = total_energy
        best_weights = individual
    
print(last_energy)
print(best_weights)
print_weights_of(best_weights)
print('This results in the following energy', last_energy)

[-9194.05261623]
[0.05122504962148345, 0.02262148245402208, 0.0007665190406000878, 0.0034207023272021875, 0.0011036484602956855, 0.004089114123908813, 7.916491128360906, 0.00028235561158206416]

Here are the weights for the following energy terms...

 bond = 0.05122504962148345
 angle = 0.02262148245402208
 dihed = 0.0007665190406000878
 nb = 0.0034207023272021875
 eel = 0.0011036484602956855
 vdw = 0.004089114123908813
 el = 7.916491128360906
 egb = 0.00028235561158206416

This results in the following energy [-9194.05261623]


In [173]:
def print_weights_of(weights):
    print()
    print("Here are the weights for the following energy terms...")
    print()
    print(" bond =", weights[0])
    print(" angle =", weights[1])
    print(" dihed =", weights[2])
    print(" nb =", weights[3])
    print(" eel =", weights[4])
    print(" vdw =", weights[5])
    print(" el =", weights[6])
    print(" egb =", weights[7])
    print()

## Left to do

1. Repeatedly add solutions to fill an initial population
2. Define the fitness function: how negative the total energy is
3. evaluate the fittest solution, i.e. the one that has the most negative energy
4. add fittest solution to the next generation (elitism)
5. tournament selection of individuals
6. mutation operator (peterb the weights slightly)
7. crossover operator, use fractions instead of just weights, to keep the total weight constant at 8
8. loop and print out fittest solution of each generation
9. plot fitness against generation number
10. return weights of fittest solution, and its total energy

# Rashid's Code

In [1]:
# # initialization of the energy and weight vectors
# evector = [49.7829, 138.4463, 217.8041, 39.1752, 691.3034, -87.4435, -1053.1789, -272.7384]
# weights = [1]*8

# # definition of the magnitude of perturbation
# pert = .001

# # perturbation of the weight vector
# def perturb(wvector, index):
#     wvector[index] += pert
#     for i in list(range(8)):
#         if index != i:
#             wvector[i] -= pert/7.
            
# # calculation of the total energy
# def sum_energies(wvector):
#     sum_components = 0
#     for i in list(range(8)):
#         sum_components += evector[i] * wvector[i]
#     return sum_components

# energy = sum_energies(weights)

# # performs perturbation until energy reaches a threshold
# while energy > -400:
#     # iterates over all elements in the weight vector
#     for i in list(range(8)):
#         # performs the perturbation at the current element
#         perturb(weights, i)
#         # if energy decreases, prints the value
#         if sum_energies(weights) < energy:
#             energy = sum_energies(weights)
#             print(energy)